論文  
https://arxiv.org/abs/2203.01941<br>
<br>  
GitHub  
https://github.com/kakaobrain/rq-vae-transformer<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/rq_vae_transformer_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 環境セットアップ

## GPU確認

In [ ]:
!nvidia-smi

## GitHubからコード取得

In [ ]:
%cd /content

!git clone https://github.com/kakaobrain/rq-vae-transformer.git

## ライブラリのインストール

In [ ]:
%cd /content/rq-vae-transformer

# もし RESTART RUNTIMEが表示されたら「ランタイム」→「ランタイムを再起動」
!pip install -r requirements.txt

## ライブラリのインポート

In [ ]:
%cd /content/rq-vae-transformer

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image
import yaml
import torch
import torchvision
import clip
import torch.nn.functional as F

from notebooks.notebook_utils import TextEncoder, load_model, get_generated_images_by_texts

## 学習済みモデルのセットアップ

In [ ]:
%cd /content/rq-vae-transformer
!mkdir pretrained
%cd pretrained

!wget https://arena.kakaocdn.net/brainrepo/models/RQVAE/dcd95e8f08408e113aab6451fae895f5/cc3m.tar.gz
!tar -xvf cc3m.tar.gz

## モデルのロード

In [ ]:
vqvae_path = '/content/rq-vae-transformer/pretrained/cc3m/stage1/model.pt'
model_path = '/content/rq-vae-transformer/pretrained/cc3m/stage2/model.pt'

# load stage 1 model: RQ-VAE
model_vqvae, _ = load_model(vqvae_path)

# load stage 2 model: RQ-Transformer
model_ar, config = load_model(model_path, ema=False)

# GPUにセット
model_ar = model_ar.cuda().eval()
model_vqvae = model_vqvae.cuda().eval()

# CLIPモデルのダウンロード
model_clip, preprocess_clip = clip.load("ViT-B/32", device='cpu')
model_clip = model_clip.cuda().eval()

# prepare text encoder to tokenize natual languages
text_encoder = TextEncoder(tokenizer_name=config.dataset.txt_tok_name, 
                           context_length=config.dataset.context_length)

# Text to Image

In [ ]:
#@markdown 入力テキストを設定してください。
text_prompts = "'a photo of \"Cherry blossoms in the snow\"'" #@param {type:"string"}

num_samples = 16
temperature= 1.0
top_k=1024
top_p=0.95

In [ ]:
pixels = get_generated_images_by_texts(model_ar,
                                       model_vqvae,
                                       text_encoder,
                                       model_clip,
                                       preprocess_clip,
                                       text_prompts,
                                       num_samples,
                                       temperature,
                                       top_k,
                                       top_p,
                                      )

## 結果の表示

In [ ]:
num_visualize_samples = 16
images = [pixel.cpu().numpy() * 0.5 + 0.5 for pixel in pixels]
images = torch.from_numpy(np.array(images[:num_visualize_samples]))
images = torch.clamp(images, 0, 1)
grid = torchvision.utils.make_grid(images, nrow=4)

img = Image.fromarray(np.uint8(grid.numpy().transpose([1,2,0])*255))
display(img)